<a href="https://colab.research.google.com/github/gracekfisher/bird-classifier/blob/main/Pretrained_34Birds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load pretrained model

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import numpy as np

In [2]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(34, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

87924736/87910968 [==============================] - 0s 0us/step


In [3]:
for layer in base_model.layers:
    layer.trainable = False

In [4]:
model.compile(optimizer='adam', metrics=['accuracy'],loss='categorical_crossentropy')

## Load googledrive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Data generators and train model

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   dtype="float32",
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255,
                                   dtype="float32")

In [7]:
train_generator = train_datagen.flow_from_directory(
    "gdrive/MyDrive/Birds/train_cut",
    target_size=(299, 299),
    batch_size=20,
    class_mode="categorical"
)

valid_generator = valid_datagen.flow_from_directory(
    "gdrive/MyDrive/Birds/test_cut",
    target_size=(299, 299),
    batch_size=20,
    class_mode="categorical"
)

Found 6668 images belonging to 34 classes.
Found 170 images belonging to 34 classes.


In [10]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=3, restore_best_weights=True, monitor='val_accuracy')

In [11]:
model.fit(train_generator, 
          steps_per_epoch=100, 
          validation_steps=10,
          validation_data=valid_generator,
          epochs=3,
          callbacks=[es]
          )

Epoch 1/3
100/100 [==============================] - 712s 7s/step - loss: 1.1572 - accuracy: 0.6820 - val_loss: 0.3407 - val_accuracy: 0.8882
Epoch 2/3
100/100 [==============================] - 471s 5s/step - loss: 0.3878 - accuracy: 0.8828
Epoch 3/3
100/100 [==============================] - 414s 4s/step - loss: 0.2731 - accuracy: 0.9145


In [12]:
model.evaluate(valid_generator)

9/9 [==============================] - 31s 3s/step - loss: 0.3339 - accuracy: 0.8882


[0.3338734805583954, 0.8882352709770203]